<a href="https://colab.research.google.com/github/windopper/NLPDeepLearningPractice/blob/main/ConvolutionNerualNetwork/analysis_IMDB_review_using_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1D CNN으로 IMDB 리뷰 분류하기

In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(x_train[:5])

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

In [4]:
max_len = 200
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [6]:
print('x_train의 크기 : ', x_train.shape)
print('x_test의 크기 : ', x_test.shape)

x_train의 크기 :  (25000, 200)
x_test의 크기 :  (25000, 200)


In [7]:
print(y_train[:5])

[1 0 0 1 0]


1D CNN 모델 설계

In [10]:
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPool1D, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 256
dropout_ratio = 0.3
num_filters = 256
kernel_size = 3
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.4001 - acc: 0.8042
Epoch 00001: val_acc improved from -inf to 0.87212, saving model to best_model.h5
782/782 [==============================] - 28s 23ms/step - loss: 0.4001 - acc: 0.8042 - val_loss: 0.2990 - val_acc: 0.8721
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.2067 - acc: 0.9209
Epoch 00002: val_acc improved from 0.87212 to 0.89120, saving model to best_model.h5
782/782 [==============================] - 17s 22ms/step - loss: 0.2067 - acc: 0.9209 - val_loss: 0.2646 - val_acc: 0.8912
Epoch 3/20
780/782 [============================>.] - ETA: 0s - loss: 0.0959 - acc: 0.9665
Epoch 00003: val_acc did not improve from 0.89120
782/782 [==============================] - 19s 24ms/step - loss: 0.0958 - acc: 0.9665 - val_loss: 0.3250 - val_acc: 0.8868
Epoch 4/20
781/782 [============================>.] - ETA: 0s - loss: 0.0400 - acc: 0.9870
Epoch 00004: val_acc did not improve from 0.89120


In [11]:
loaded_model = load_model('best_model.h5')
print('\n테스트 정확도 : ', (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 5s 6ms/step - loss: 0.2646 - acc: 0.8912

테스트 정확도 :  0.8912000060081482
